## [필수 분석]

입점 기업별 RFM 분석(3그룹) 진행

매출 시각화

    -월별, 연도별 매출

    -월별 순수익 (처리 상황, 할부기간
고려)

결제 방법에 따른 분석

결제 방법과 결제금액의 연관성
분석

결제방법은 맨 앞의 한가지만
사용한 것으로 간주


### [선택 분석]

매출 시각화 2

최대 매출 상품 3종류 집계

주문 연도에 따른 해당 상품의
매출 증감 분석

연관성 분석

주문한 달과 판매금액의
상관관계 분석

※ 연도의 변화는 무시한다

In [ ]:
import pandas as pd

df = pd.read_excel('../../Downloads/미니프로젝트-쇼핑몰 실습데이터.xlsx')
# '판매금액', '업체명', '주문일자' 열에 대한 결측치를 모두 제거
df2 = df.dropna(subset=['판매금액', '업체명', '주문일자', '결제방법'])
# '지니 태블릿(후불집행)'을 '지니'로 변경
df2.loc[df2['업체명'] == '지니 태블릿(후불집행)', '업체명'] = '지니'

# '지니 태블릿'을 '지니'로 변경
df2.loc[df2['업체명'] == '지니 태블릿', '업체명'] = '지니'

In [ ]:
import pandas as pd


# '결제방법' 열 값 변경
df2['결제방법'] = df2['결제방법'].replace({'가상계좌':'현금','무통장입금': '현금','현금간편결제': '현금','웰컴마일':'적립금'})

In [ ]:
# '결제방법' 열에서 앞글자만 추출하여 새로운 열 만들기
df2['앞글자'] = df2['결제방법'].apply(lambda x: x[0] if isinstance(x, str) and len(x) > 0 else None)

# 결과 출력
print(df2[['결제방법', '앞글자']])

# '결제방법'이 "-"인 행 제거
df2 = df2[df2['앞글자'] != ' ']
df2 = df2[df2['앞글자'] != '-']
df2 = df2[df2['앞글자'] != '정']
df2 = df2[df2['앞글자'] != '후']

# 결과 출력
df2['앞글자'].unique()

In [ ]:
df2['앞글자'] = df2['앞글자'].replace('신', '신용카드')
df2['앞글자'] = df2['앞글자'].replace('현', '현금')
df2['앞글자'] = df2['앞글자'].replace('적', '적립금')
df2['앞글자'] = df2['앞글자'].replace('포', '포인트')

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'

# 결제방법별로 금액 합산
payment = df2.groupby('앞글자')['판매금액'].sum()

# 파이 차트 그리기
custom_colors = ['blue','green','red','orange']
plt.pie(payment, labels=payment.index, autopct='%1.1f%%', startangle=30, colors=custom_colors)
plt.title('결제방법별 금액 분포')
plt.show()

In [ ]:
# 결제방법별 판매금액 평균 계산
average_payment = df2.groupby('앞글자')['판매금액'].mean().reset_index()

# 막대그래프 그리기
custom_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
plt.bar(average_payment['앞글자'], average_payment['판매금액'], color=custom_colors)
plt.title('결제방법별 평균 판매금액')
plt.xlabel('결제방법')
plt.ylabel('평균 금액')
plt.show()

In [ ]:
# 각 결제방법별로 상위 10개 행 추출
top_10_by_payment_method = df2.groupby('앞글자').apply(lambda group: group.nlargest(1000, '판매금액')).reset_index(drop=True)

# 각 결제방법별 '판매금액'의 평균 계산
average_sales_by_payment_method = top_10_by_payment_method.groupby('앞글자')['판매금액'].mean().reset_index()

# 결과 출력
print(average_sales_by_payment_method)


In [ ]:
# 판매금액이 0이 아닌 데이터프레임 생성
nz_df2 = df2[df2['판매금액'] != 0]

# 각 결제방법별로 하위 10개 행 추출
bottom_10_by_payment_method = nz_df2.groupby('앞글자').apply(lambda group: group.nsmallest(1000, '판매금액')).reset_index(drop=True)

# 각 결제방법별 '판매금액'의 평균 계산
average_sales_bottom_10_by_payment_method = bottom_10_by_payment_method.groupby('앞글자')['판매금액'].mean().reset_index()

# 결과 출력
print(average_sales_bottom_10_by_payment_method)